In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [1]:
import warnings
warnings.filterwarnings('ignore')

# all import statements
import numpy as np
import pandas as pd
import pydicom as pyd
import os
import matplotlib.pyplot as plt
import mudicom

from os.path import dirname, join
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir
from PIL import Image

from scipy.misc import imresize
from scipy.signal import convolve2d
import scipy
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries

from numpy import newaxis

from skimage.measure import shannon_entropy
from skimage import io, color, img_as_ubyte
from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics.cluster import entropy


import math
from skimage.measure import label, regionprops

from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb

import matplotlib.patches as mpatches

import statistics

from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max

import cv2
from skimage import img_as_float
from skimage import exposure

In [2]:
import pickle

from tqdm import tqdm
import os
import sys

In [3]:
def load_pickle(fileName):
    with open(fileName, "rb") as fp:
        file = pickle.load(fp)
    return file 

def write_pickle(file, fileName):
    with open(fileName, "wb") as fp:
        pickle.dump(file, fp)

### GLCM

In [4]:

def s_entropy(dcm):
    return shannon_entropy(dcm)

def entropy_simple(dcm):
    return entropy(dcm)

class glcm:
    def __init__(self, image):
        distance = [1, 2, 3]
        angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
        self.image = img_as_ubyte(image.astype('int64'))
        self.glcm_mat = greycomatrix(self.image, distances = distance, angles = angles, symmetric = True, normed = True)
        self.properties = ['correlation', 'homogeneity', 'contrast', 'energy']
            
    def correlation(self):
        return greycoprops(self.glcm_mat, 'correlation').flatten()
    
    def homogeneity(self):
        return greycoprops(self.glcm_mat, 'homogeneity').flatten()
    
    def contrast(self):
        return greycoprops(self.glcm_mat, 'contrast').flatten()
    
    def energy(self):
        return greycoprops(self.glcm_mat, 'energy').flatten()
    
    def glcm_all(self):
        return np.hstack([greycoprops(self.glcm_mat, props).ravel() for props in self.properties])
    

### Region Props

In [5]:
# thresh_ada = threshold_otsu(image_adahist)
# bw_ada = closing(image_adahist > thresh_ada, square(3))
# bw_clear_ada = clear_border(bw_ada)

class region_props:
    def __init__(self, image):
        self.image = image
        self.thresh = threshold_otsu(self.image)
        self.bw = closing(self.image > self.thresh, square(3))
        self.bw_clear = clear_border(self.bw)
        self.bw_label = label(self.bw_clear)
        self.regions = regionprops(self.bw_label)
#         self.idx = 0
        
    def plot_image_bw(self):
        plt.imshow(self.bw, cmap = plt.cm.bone)
    
    def plot_image(self):
        plt.imshow(self.bw_clear, cmap=plt.cm.bone)
        
    def plot_image_with_labels(self):
        plt.imshow(self.bw_label, cmap=plt.cm.bone)
        
    def max_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return max(lista)
    
    def eccentricity(self):
        
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return self.regions[idx].eccentricity
    
    def euler_number(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return self.regions[idx].euler_number
    
    def solidity(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return self.regions[idx].solidity
    
    def perimeter(self):
        lista = []
        for e in self.regions:
            lista.append(e.perimeter)
        return max(lista)
    
    def mean_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        return statistics.mean(lista)
    
    def std_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>2:
            return statistics.stdev(lista)
        else:
            return -1
    
    def thresh_img(self):
        return self.thresh
        

### Moments

In [6]:
class moments:
    
    def __init__(self, image):
        self.image = image
        self.moment = cv2.moments(self.image)
        self.hu = cv2.HuMoments(self.moment)
        
    def get_moments(self):
#         keys = [key for key in self.moment.keys()]
        values = [value for value in self.moment.values()]
        return values
    
    def get_HuMoments(self):
        moments_hu = []
        for m in range(len(self.hu)):
            moments_hu.append(self.hu[m][0])
        return moments_hu

#### DataFrame dcm_1

In [7]:
dcm_5 = load_pickle("dcm-5")
pat_5 = load_pickle("pat-5")

In [9]:
len(dcm_5), len(pat_5)

(4000, 4000)

In [8]:
data_dcm_5 = dict()
for dc in tqdm(range(len(dcm_5))):
    data_dcm_5[pat_5[dc]] = [s_entropy(dcm_5[dc]), entropy_simple(dcm_5[dc]), moments(dcm_5[dc]).get_moments(), moments(dcm_5[dc]).get_HuMoments(),
                      region_props(dcm_5[dc]).max_area(), region_props(dcm_5[dc]).eccentricity(), region_props(dcm_5[dc]).euler_number(),
                      region_props(dcm_5[dc]).solidity(), region_props(dcm_5[dc]).perimeter(), region_props(dcm_5[dc]).mean_area(), 
                      region_props(dcm_5[dc]).std_area(), region_props(dcm_5[dc]).thresh_img(), glcm(dcm_5[dc]).correlation(), glcm(dcm_5[dc]).homogeneity(),
                      glcm(dcm_5[dc]).contrast(), glcm(dcm_5[dc]).energy()]

100%|██████████| 4000/4000 [1:10:12<00:00,  1.14it/s]


In [9]:
df_dcm_5 = pd.DataFrame(data_dcm_5).T

In [10]:
df_dcm_5.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
cd48389f-d5e7-4f7e-b0e0-dda7a74b9445,7.3496,5.09435,"[134380633.0, 70710615429.0, 75880251046.0, 45...","[0.0010601197686282398, 1.2925638740571287e-08...",743,0.957856,0,0.562453,329.409,34,94.784,107,"[0.9934056936018071, 0.9890499172173415, 0.994...","[0.5023112176484367, 0.4456608445021911, 0.549...","[58.28542735825998, 96.58757664622766, 45.7742...","[0.08223717784285099, 0.08060640527906413, 0.0..."
17e4fe87-199a-481d-a77f-52a4e481780a,7.43391,5.1528,"[117780332.0, 63013966281.0, 66789646090.0, 42...","[0.0013351941878377484, 1.848744067931869e-08,...",582,0.95912,1,0.435955,268.078,21,63.4508,109,"[0.9880133058990229, 0.975536973082404, 0.9855...","[0.4217518390027662, 0.34044676821865993, 0.41...","[63.32953781769306, 128.88363915381223, 76.317...","[0.04178702593724511, 0.039204088990375224, 0...."
f93481d4-8870-429b-9293-333d5f82a636,7.7966,5.40419,"[130319499.0, 69594851873.0, 69817436397.0, 46...","[0.001243923576016914, 2.075410332971951e-08, ...",108,0.746112,1,0.662577,55.3848,5,15.5563,119,"[0.9988169040460481, 0.9978031467456753, 0.998...","[0.41779668676678566, 0.34651889880181, 0.4348...","[10.833634989002933, 20.09191431866676, 11.461...","[0.036042215146370724, 0.034144022544909204, 0..."
ed369fe4-b5e2-4ac4-8bef-ac7f5e18042b,7.29084,5.05362,"[105928059.0, 52237660066.0, 65030811044.0, 31...","[0.0012241425543837252, 2.4313919189315562e-08...",601,0.901268,1,0.682955,162.332,34,105.835,98,"[0.9977212044154437, 0.9961869714496263, 0.998...","[0.5910807326535219, 0.5352899627191908, 0.656...","[21.030207569648113, 35.19122260348256, 14.567...","[0.114880290264844, 0.11410369561643531, 0.117..."
60bab750-3d16-4e10-ae2d-5f4cf93ec424,7.61082,5.27542,"[152633051.0, 77822939647.0, 90567696427.0, 51...","[0.001002117689660007, 1.7713030518635153e-09,...",497,0.92868,0,0.63555,233.841,20,66.1438,125,"[0.9969174039723151, 0.9940368147329766, 0.996...","[0.5429338887874084, 0.48616534381820964, 0.57...","[32.57556951826734, 62.96210520683134, 34.7437...","[0.07426449226014069, 0.0722611799893658, 0.07..."


In [11]:
df_dcm_5.to_pickle('df_dcm_5')

In [12]:
df_dcm_5.to_csv('df_dcm_5.csv')